# Python ChainMap

A **ChainMap** is a class from the collections module that provides the ability to link multiple mappings together such that they end up being a single unit.

If you look at the documentation, you will notice that it accepts `*maps`, which means that a ChainMap will accept any number of mappings or dictionaries and turn them into a single view that you can update.

In [2]:
from collections import ChainMap
car_parts = {'hood': 500, 'engine': 5000, 'front_door': 750}
car_options = {'A/C': 1000, 'Turbo': 2500, 'rollbar': 300}
car_accessories = {'cover': 100, 'hood_ornament': 150, 'seat_cover': 99}
car_pricing = ChainMap(car_accessories, car_options, car_parts)
car_pricing['hood']

500

This is especially useful if you want to set up defaults.

Let’s pretend that we want to create an application that has some defaults. The application will also be aware of the operating system’s environment variables. If there is an environment variable that matches one of the keys that we are defaulting to in our application, the environment will override our default. Let’s further pretend that we can pass arguments to our application. These arguments take precedence over the environment and the defaults. This is one place where a ChainMap can really shine. 

In [3]:
import argparse
import os
from collections import ChainMap
def main():
   app_defaults = {'username':'admin', 'password':'admin'}
   parser = argparse.ArgumentParser()
   parser.add_argument('-u', '--username')
   parser.add_argument('-p', '--password')
   args = parser.parse_args()
   command_line_arguments = {key:value for key, value
                             in vars(args).items() if value}
   chain = ChainMap(command_line_arguments, os.environ,
                    app_defaults)
   print(chain['username'])
if __name__ == '__main__':
   main()
   os.environ['username'] = 'test'
   main()


usage: ipykernel_launcher.py [-h] [-u USERNAME] [-p PASSWORD]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-c25d10c9-3691-4013-9274-5c81664a44d7.json


SystemExit: 2

/home/muhammadtaqi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Let’s break this down a little. Here we import Python’s argparse module along with the os module. We also import ChainMap. Next, we have a simple function that has some silly defaults. I’ve seen these defaults used for some popular routers. Then we set up our argument parser and tell it how to handle certain command line options. You will notice that argparse doesn’t provide a way to get a dictionary object of its arguments, so we use a dict comprehension to extract what we need. The other cool piece here is the use of Python’s built-in vars. If you were to call it without an argument, vars would behave like Python’s built-inlocals. But if you do pass in an object, then vars is the equivalent to object’s __dict__ property.
In other words, vars(args) equals args.__dict__. Finally, create our ChainMap by passing in our command line arguments (if there are any), then the environment variables, and finally the defaults. At the end of the code, we try calling our function, then setting an environment variable and calling it again. Give it a try and you’ll see that it prints out admin and then test as expected. Now, let’s try calling the script with a command line argument:


python chain_map.py -u mike

When I ran this, I got mike back twice. This is because our command line argument overrides everything else. It doesn’t matter that we set the environment because our ChainMap will look at the command line arguments first before anything else.
Wrapping Up
Now you now what a ChainMap is and how you might use it. I found them to be quite interesting and I think I already have a use-case or two for them that I hope to be implementing soon.


## ChainMap — search multiple dictionaries 

ChainMap — search multiple dictionaries
The ChainMap class manages a sequence of dictionaries, searching them in order to find values associated with keys. A ChainMap makes a good “context” container, since it can be treated as a stack with changes happening as the stack grows, then being discarded again as the stack shrinks.
Accessing Values
The ChainMap supports the same API as a regular dictionary for accessing existing values.
collections_chainmap_read.py
import collections

a = {'a': 'A', 'c': 'C'}
b = {'b': 'B', 'c': 'D'}

m = collections.ChainMap(a, b)

print('Individual Values')
print('a = {}'.format(m['a']))
print('b = {}'.format(m['b']))
print('c = {}'.format(m['c']))
print()

print('Keys = {}'.format(list(m.keys())))
print('Values = {}'.format(list(m.values())))
print()

print('Items:')
for k, v in m.items():
    print('{} = {}'.format(k, v))
print()

print('"d" in m: {}'.format(('d' in m)))


The child mappings are searched in the order they are passed to the constructor, so the value reported for the key 'c' comes from the a dictionary.
$ python3 collections_chainmap_read.py

Individual Values
a = A
b = B
c = C

Keys = ['c', 'b', 'a']
Values = ['C', 'B', 'A']

Items:
c = C
b = B
a = A

"d" in m: False


Reordering
The ChainMap stores the list of mappings over which it searches in a list in its maps attribute. The list is mutable, so it is possible to add new mappings directly or to change the order of the elements to control look-up and update behavior.
collections_chainmap_reorder.py
import collections

a = {'a': 'A', 'c': 'C'}
b = {'b': 'B', 'c': 'D'}

m = collections.ChainMap(a, b)

print(m.maps)
print('c = {}\n'.format(m['c']))

# reverse the list
m.maps = list(reversed(m.maps))

print(m.maps)
print('c = {}'.format(m['c']))


When the list of mappings is reversed, the value associated with 'c' changes.
$ python3 collections_chainmap_reorder.py

[{'c': 'C', 'a': 'A'}, {'c': 'D', 'b': 'B'}]
c = C

[{'c': 'D', 'b': 'B'}, {'c': 'C', 'a': 'A'}]
c = D


Updating Values
A ChainMap does not cache the values in the child mappings, so if their contents are modified the results are reflected when the ChainMap is accessed.
collections_chainmap_update_behind.py
import collections

a = {'a': 'A', 'c': 'C'}
b = {'b': 'B', 'c': 'D'}

m = collections.ChainMap(a, b)
print('Before: {}'.format(m['c']))
a['c'] = 'E'
print('After : {}'.format(m['c']))


Changing the values associated with existing keys and adding new elements works the same way.
$ python3 collections_chainmap_update_behind.py

Before: C
After : E


It is also possible to set values through the ChainMap directly, though only the first mapping in the chain is actually modified.
collections_chainmap_update_directly.py
import collections

a = {'a': 'A', 'c': 'C'}
b = {'b': 'B', 'c': 'D'}

m = collections.ChainMap(a, b)
print('Before:', m)
m['c'] = 'E'
print('After :', m)
print('a:', a)


When the new value is stored using m, the a mapping is updated.
$ python3 collections_chainmap_update_directly.py

Before: ChainMap({'c': 'C', 'a': 'A'}, {'c': 'D', 'b': 'B'})
After : ChainMap({'c': 'E', 'a': 'A'}, {'c': 'D', 'b': 'B'})
a: {'c': 'E', 'a': 'A'}


ChainMap provides a convenience method for creating a new instance with one extra mapping at the front of the maps list to make it easy to avoid modifying the existing underlying data structures.
collections_chainmap_new_child.py
import collections

a = {'a': 'A', 'c': 'C'}
b = {'b': 'B', 'c': 'D'}

m1 = collections.ChainMap(a, b)
m2 = m1.new_child()

print('m1 before:', m1)
print('m2 before:', m2)

m2['c'] = 'E'

print('m1 after:', m1)
print('m2 after:', m2)


This stacking behavior is what makes it convenient to use ChainMap instances as template or application contexts, since it is easy to add or update values in one iteration, then discard the changes for the next iteration.
$ python3 collections_chainmap_new_child.py

m1 before: ChainMap({'c': 'C', 'a': 'A'}, {'c': 'D', 'b': 'B'})
m2 before: ChainMap({}, {'c': 'C', 'a': 'A'}, {'c': 'D', 'b': 'B
'})
m1 after: ChainMap({'c': 'C', 'a': 'A'}, {'c': 'D', 'b': 'B'})
m2 after: ChainMap({'c': 'E'}, {'c': 'C', 'a': 'A'}, {'c': 'D',
'b': 'B'})


For situations where the new context is known or built in advance it is also possible to pass a mapping to new_child().
collections_chainmap_new_child_explicit.py
import collections

a = {'a': 'A', 'c': 'C'}
b = {'b': 'B', 'c': 'D'}
c = {'c': 'E'}

m1 = collections.ChainMap(a, b)
m2 = m1.new_child(c)

print('m1["c"] = {}'.format(m1['c']))
print('m2["c"] = {}'.format(m2['c']))


This is the equivalent of
m2 = collections.ChainMap(c, *m1.maps)


and produces
$ python3 collections_chainmap_new_child_explicit.py

m1["c"] = C
m2["c"] = E


## Reference 

    1. https://docs.python.org/3/library/collections.html#chainmap-objects